In [ ]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# Mod
import tensorflow as tf

# __file__ does not work on Jupyter notebook,using Path to identify notebook's directory
from pathlib import Path

# prevent tensorflow gpus' memory allocation log
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Define a flask app
app = Flask(__name__)

#saved model w Keras model.save()
MODEL_PATH = 'models/your_model.h5'

from keras.applications.resnet50 import ResNet50
model = ResNet50(weights='imagenet')
graph = tf.get_default_graph() # Change
print('Model loaded. Check http://127.0.0.1:8000/')


def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))

    #preprocessing the image to array
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x, mode='caffe')
    with graph.as_default(): 
        preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    # Index page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get file image from POST request
        f = request.files['image']

        # Save the file to /uploads
        basepath = Path.cwd()
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        
        #make predictions
        preds = model_predict(file_path, model)
        # ImageNet Decode
        pred_class = decode_predictions(preds, top=1)   
        # Convert to string
        result = str(pred_class[0][0][1])              
        return result
    return None


if __name__ == '__main__':
    # Serve the app with gevent
    http_server = WSGIServer(('0.0.0.0', 8000), app)
    http_server.serve_forever()